In [1]:
pip install yfinance

# Long Term Investment Strategy Analysis

In [2]:
# initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import os
import requests

import yfinance as yf
import pytz
import sys

# Load and clean the data

In [3]:
#### World Stock prices

# Get the directory path of the csv file and select only US stocks
script_directory = os.path.dirname(os.path.abspath("Resources/World-Stock-Prices-Dataset.csv"))
worldstockcsv = os.path.join(script_directory, "World-Stock-Prices-Dataset.csv")
world_stock = pd.read_csv(worldstockcsv)
us_stock_prices_df = world_stock[world_stock['Country'] == 'usa']

#Filter data for only user's selected stocks
Risk_Evaluation = %run -i Risk_Evaluation.ipynb
us_stock_prices_df = us_stock_prices_df[us_stock_prices_df['Ticker'].isin(risk_tickers)]

# Drop the time value in the Date column
us_stock_prices_df['Date'] = pd.to_datetime(us_stock_prices_df['Date'], utc=True)
us_stock_prices_df['Date'] = us_stock_prices_df['Date'].dt.normalize()
us_stock_prices_df['Date'] = us_stock_prices_df['Date'].dt.strftime('%Y-%m-%d')

# Drop rows with null values
us_stock_prices_df.dropna(inplace=True)

# Drop 'Dividends' and 'Stock Splits' columns as they are empty
us_stock_prices_df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)


#### S&P 500 - ^GSPC

# Define the ticker symbol
ticker = "^GSPC"

# Set the date range for historical data
start_date = "2003-10-01"
end_date = "2023-10-01"

# Fetch historical data from Yahoo Finance
SP_df = yf.download(ticker, start=start_date, end=end_date)

#Reset the index
SP_df.reset_index(inplace=True)

# Create a column called ticker
SP_df['Ticker'] = 'S&P 500'

Your choice of 'Trade Term' will define the timeframe with which we will analyze stock returns
Select a trade term that matches your current investment goals for collecting returns from stock investments

Choose a Trade Term From These Options:
1. 1Yrs (1 Year)
2. 5Yrs (5 Years)
3. 10Yrs (10 Years)
4. 15Yrs (15 Years)
5. 7Dys (7 Days)
6. 21Dys (21 Days)
7. 40Dys (40 Days)




Enter your trade term 10Yrs




Your choice of 'Risk Threshold' will define how risky and volatile the stocks in the final portfolio will be
Select a risk threshold that matches your comfort levels when it comes to risk

Choose a Risk Threshold:
1. high
2. mid
3. low




Enter your risk threshold low


C:\Users\Oluwa\AppData\Local\Temp\ipykernel_22552\678915812.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Smaller_Dataframe['Date'] = Smaller_Dataframe['Date'].astype(str)


[*********************100%%**********************]  1 of 1 completed


In [4]:
# Combine the two DataFrames into one
columns_to_concat = ['Date', 'Close', 'Open', 'High', 'Low', 'Ticker', 'Volume']

# Select the specified columns from each DataFrame
us_stock_prices_subset = us_stock_prices_df[columns_to_concat]
SP_subset = SP_df[columns_to_concat]

# Concatenate the selected columns
combined_df = pd.concat([us_stock_prices_subset, SP_subset], axis=0)

# Sort the index to ensure it's in date order
combined_df.sort_index(inplace=True)

# Ensure 'Date' column is in datetime format
combined_df['Date'] = pd.to_datetime(combined_df['Date'])

# Reformat the 'Date' column to 'YYYY-MM-DD'
combined_df['Date'] = combined_df['Date'].dt.strftime('%Y-%m-%d')

combined_df

,Date,Close,Open,High,Low,Ticker,Volume
0,2023-09-20,4.670000,4.840000,4.910000,4.630000,PTON,7.441900e+06
0,2003-10-01,1018.219971,995.969971,1018.219971,995.969971,S&P 500,1.566300e+09
1,2003-10-02,1020.239990,1018.219971,1021.869995,1013.380005,S&P 500,1.269300e+09
2,2003-10-03,1029.849976,1020.239990,1039.310059,1020.239990,S&P 500,1.570500e+09
3,2003-10-06,1034.349976,1029.849976,1036.479980,1029.150024,S&P 500,1.025800e+09
...,...,...,...,...,...,...,...
267836,2023-08-29,15.670000,15.550000,15.790000,15.440000,JWN,6.393800e+06
267837,2023-08-30,15.690000,15.600000,15.870000,15.540000,JWN,5.777300e+06
267838,2023-08-31,16.219999,15.760000,16.250000,15.680000,JWN,4.982800e+06
267839,2023-09-01,16.070000,16.320000,16.530001,16.010000,JWN,3.604400e+06


# SMA

In [5]:
# Select only columns 'Ticker' and 'Close' from the existing DataFrame
closing_prices_df = combined_df[['Date', 'Ticker', 'Close']]

# Set 'Date' column as the index
closing_prices_df.set_index('Date', inplace=True)

# Convert the index to datetime and extract the date component
closing_prices_df.index = pd.to_datetime(closing_prices_df.index, format='%Y-%m-%d')
closing_prices_df.index = closing_prices_df.index.date
closing_prices_df

,Ticker,Close
2023-09-20,PTON,4.670000
2003-10-01,S&P 500,1018.219971
2003-10-02,S&P 500,1020.239990
2003-10-03,S&P 500,1029.849976
2003-10-06,S&P 500,1034.349976
...,...,...
2023-08-29,JWN,15.670000
2023-08-30,JWN,15.690000
2023-08-31,JWN,16.219999
2023-09-01,JWN,16.070000


In [6]:
# Define the SMA window (5 years)
sma_window = 1260

# Create a list to store DataFrames for each ticker
sma_dataframes = []

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMA for the ticker using the rolling mean and assign it using .loc
    ticker_data.loc[:, f'SMA_{sma_window}'] = ticker_data['Close'].rolling(window=sma_window).mean()
    
    # Get the latest SMA value for the ticker
    latest_sma = ticker_data.iloc[-1][f'SMA_{sma_window}']
    
    # Create a DataFrame for the ticker and its latest SMA value
    ticker_sma_df = pd.DataFrame({'Ticker': [ticker], f'SMA_{sma_window}': [latest_sma]})
    
    # Append the DataFrame to the list
    sma_dataframes.append(ticker_sma_df)

# Concatenate the list of DataFrames into sma_summary_df
sma_summary_df = pd.concat(sma_dataframes, ignore_index=True)

# Sort sma_summary_df by SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_window}', ascending=False, inplace=True)

#Change SMA Index
sma_summary_df.set_index("Ticker", inplace = True)

# Select the top 10 tickers with the best SMA values
top_10_sma2520_tickers = sma_summary_df.head(10)

# Display the top 10 tickers with the best SMA values
print("Top 10 tickers with the best 5 year SMA values:")
print(top_10_sma2520_tickers)



Top 10 tickers with the best 5 year SMA values:
            SMA_1260
Ticker              
S&P 500  3666.196245
V         194.710504
HSY       165.526447
TSLA      156.435700
JNJ       144.863970
SQ        116.184270
AMD        72.152381
CROX       69.421301
JWN        26.889125
PTON             NaN


In [7]:
# Define the SMA window (10 years)
sma_window = 2520

# Create a list to store DataFrames for each ticker
sma_dataframes = []

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMA for the ticker using the rolling mean and assign it using .loc
    ticker_data.loc[:, f'SMA_{sma_window}'] = ticker_data['Close'].rolling(window=sma_window).mean()
    
    # Get the latest SMA value for the ticker
    latest_sma = ticker_data.iloc[-1][f'SMA_{sma_window}']
    
    # Create a DataFrame for the ticker and its latest SMA value
    ticker_sma_df = pd.DataFrame({'Ticker': [ticker], f'SMA_{sma_window}': [latest_sma]})
    
    # Append the DataFrame to the list
    sma_dataframes.append(ticker_sma_df)

# Concatenate the list of DataFrames into sma_summary_df
sma_summary_df = pd.concat(sma_dataframes, ignore_index=True)

# Sort sma_summary_df by SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_window}', ascending=False, inplace=True)

# Select the top 10 tickers with the best SMA values
top_10_sma5040_tickers = sma_summary_df.head(10)

# Display the top 10 tickers with the best SMA values
print("Top 10 tickers with the best 10 year SMA values:")
print(top_10_sma5040_tickers)


Top 10 tickers with the best 10 year SMA values:
     Ticker     SMA_2520
1   S&P 500  2936.420428
7         V   135.835090
3       HSY   124.676747
9       JNJ   118.402308
2      TSLA    86.497212
5      CROX    40.675032
6       AMD    39.491020
10      JWN    35.981680
0      PTON          NaN
4      PINS          NaN


In [8]:
# Define the SMA window (15 years)
sma_window = 3780

# Create a list to store DataFrames for each ticker
sma_dataframes = []

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMA for the ticker using the rolling mean and assign it using .loc
    ticker_data.loc[:, f'SMA_{sma_window}'] = ticker_data['Close'].rolling(window=sma_window).mean()
    
    # Get the latest SMA value for the ticker
    latest_sma = ticker_data.iloc[-1][f'SMA_{sma_window}']
    
    # Create a DataFrame for the ticker and its latest SMA value
    ticker_sma_df = pd.DataFrame({'Ticker': [ticker], f'SMA_{sma_window}': [latest_sma]})
    
    # Append the DataFrame to the list
    sma_dataframes.append(ticker_sma_df)

# Concatenate the list of DataFrames into sma_summary_df
sma_summary_df = pd.concat(sma_dataframes, ignore_index=True)

# Sort sma_summary_df by SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_window}', ascending=False, inplace=True)

# Select the top 10 tickers with the best SMA values
top_10_sma3780_tickers = sma_summary_df.head(10)

# Display the top 10 tickers with the best SMA values
print("Top 10 tickers with the best 15 year SMA values:")
print(top_10_sma3780_tickers)

Top 10 tickers with the best 15 year SMA values:
     Ticker     SMA_3780
1   S&P 500  2367.846642
7         V    97.991672
3       HSY    97.111932
9       JNJ    94.213306
10      JWN    33.408286
5      CROX    31.398667
6       AMD    28.189087
0      PTON          NaN
2      TSLA          NaN
4      PINS          NaN


In [9]:
# Define the SMA windows
sma_windows = [1260, 2520, 3780]  # Corresponding to 5, 10, and 15 years

# Create a list to store DataFrames for each ticker
sma_dataframes = []

# Loop through each unique ticker in your DataFrame
for ticker in closing_prices_df['Ticker'].unique():
    # Filter the DataFrame to get data for the current ticker and create a copy
    ticker_data = closing_prices_df[closing_prices_df['Ticker'] == ticker].copy()
    
    # Calculate the SMAs for the ticker using the rolling mean and assign them using .loc
    for window in sma_windows:
        column_name = f'SMA_{window}'
        ticker_data.loc[:, column_name] = ticker_data['Close'].rolling(window=window).mean()
    
    # Get the latest SMA values for the ticker
    latest_sma_values = [ticker_data.iloc[-1][f'SMA_{window}'] for window in sma_windows]
    
    # Create a DataFrame for the ticker and its latest SMA values
    ticker_sma_df = pd.DataFrame({'Ticker': [ticker], **{f'SMA_{window}': [value] for window, value in zip(sma_windows, latest_sma_values)}})
    
    # Append the DataFrame to the list
    sma_dataframes.append(ticker_sma_df)

# Concatenate the list of DataFrames into sma_summary_df
sma_summary_df = pd.concat(sma_dataframes, ignore_index=True)

# Sort sma_summary_df by 10-year (2520) SMA values in descending order
sma_summary_df.sort_values(by=f'SMA_{sma_windows[1]}', ascending=False, inplace=True)

# Display the DataFrame with 5, 10, and 15-year SMAs for each ticker
print("Top 10 tickers with the best 5, 10, and 15-year SMA values:")
sma_summary_top10 = sma_summary_df.head(10)
print(sma_summary_top10)


Top 10 tickers with the best 5, 10, and 15-year SMA values:
     Ticker     SMA_1260     SMA_2520     SMA_3780
1   S&P 500  3666.196245  2936.420428  2367.846642
7         V   194.710504   135.835090    97.991672
3       HSY   165.526447   124.676747    97.111932
9       JNJ   144.863970   118.402308    94.213306
2      TSLA   156.435700    86.497212          NaN
5      CROX    69.421301    40.675032    31.398667
6       AMD    72.152381    39.491020    28.189087
10      JWN    26.889125    35.981680    33.408286
0      PTON          NaN          NaN          NaN
4      PINS          NaN          NaN          NaN


In [10]:
sma_summary_top10.set_index("Ticker", inplace = True)
sma_summary_top10.hvplot(
    xlabel="Ticker", 
    ylabel="SMA Values",
    title='SMA Values (5, 10 and 15 years)',
    width = 1000
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

# Sharpe Ratios

In [11]:
# 5 year Sharp Ratios by stock

# Convert the index to datetime
combined_df.index = pd.to_datetime(combined_df.index)

# Calculate returns over the 5 years
combined_df['Returns_5Y'] = combined_df.groupby('Ticker')['Close'].pct_change(5 * 252)

# Calculate the Sharpe Ratio for each ticker's returns
sharpe_ratios = combined_df.groupby('Ticker').apply(
    lambda group: (group['Returns_5Y'].mean() / group['Returns_5Y'].std()) * np.sqrt(252)
)
sharpe_ratios = sharpe_ratios.rename("Sharpe Ratio (5 years)").reset_index()

# Drop any rows with NaN values 
sharpe_ratios.dropna(subset=['Sharpe Ratio (5 years)'], inplace=True)

# Create a new DataFrame 'sharpe_ratios' to store the results
sharpe_ratios_5y = sharpe_ratios.copy()

# Sory by ascending
sharpe_ratios_5y = sharpe_ratios_5y.sort_values(by='Sharpe Ratio (5 years)', ascending=False)

# Print or further analyze the Sharpe Ratios per ticker
print(sharpe_ratios_5y)


     Ticker  Sharpe Ratio (5 years)
10        V               46.903904
3       JNJ               25.256420
9      TSLA               21.383983
2       HSY               20.011142
7   S&P 500               18.262450
8        SQ               15.445223
1      CROX                9.717318
4       JWN                9.501606
0       AMD                6.953113


In [12]:
# 10 year Sharp Ratios by stock

# Calculate returns over the 10 years
combined_df['Returns_10Y'] = combined_df.groupby('Ticker')['Close'].pct_change(10 * 252)

# Calculate the Sharpe Ratio for each ticker's returns
sharpe_ratios = combined_df.groupby('Ticker').apply(
    lambda group: (group['Returns_10Y'].mean() / group['Returns_10Y'].std()) * np.sqrt(252)
)
sharpe_ratios = sharpe_ratios.rename("Sharpe Ratio (10 years)").reset_index()

# Drop any rows with NaN values 
sharpe_ratios.dropna(subset=['Sharpe Ratio (10 years)'], inplace=True)

# Create a new DataFrame 'sharpe_ratios' to store the results
sharpe_ratios_10y = sharpe_ratios.copy()

# Sory by ascending
sharpe_ratios_10y = sharpe_ratios_10y.sort_values(by='Sharpe Ratio (10 years)', ascending=False)


# Print or further analyze the Sharpe Ratios per ticker
print(sharpe_ratios_10y)


     Ticker  Sharpe Ratio (10 years)
10        V                51.466142
2       HSY                39.191539
9      TSLA                37.791009
3       JNJ                35.474155
7   S&P 500                31.076942
4       JWN                14.234929
1      CROX                11.988586
0       AMD                 8.541401


In [13]:
# 15 year Sharp Ratios by stock

# Calculate returns over the 15 years
combined_df['Returns_15Y'] = combined_df.groupby('Ticker')['Close'].pct_change(15 * 252)

# Calculate the Sharpe Ratio for each ticker's returns
sharpe_ratios = combined_df.groupby('Ticker').apply(
    lambda group: (group['Returns_15Y'].mean() / group['Returns_15Y'].std()) * np.sqrt(252)
)
sharpe_ratios = sharpe_ratios.rename("Sharpe Ratio (15 years)").reset_index()

# Drop any rows with NaN values 
sharpe_ratios.dropna(subset=['Sharpe Ratio (15 years)'], inplace=True)

# Create a new DataFrame 'sharpe_ratios' to store the results
sharpe_ratios_15y = sharpe_ratios.copy()

# Sory by ascending
sharpe_ratios_15y = sharpe_ratios_15y.sort_values(by='Sharpe Ratio (15 years)', ascending=False)

# Print or further analyze the Sharpe Ratios per ticker
print(sharpe_ratios_15y)

     Ticker  Sharpe Ratio (15 years)
3       JNJ                85.049696
7   S&P 500                83.952682
10        V                43.647353
2       HSY                37.633397
1      CROX                17.349017
4       JWN                15.235803
0       AMD                 9.103983


In [14]:
# Merge the DataFrames based on the 'Ticker' column to show 5, 10 and 15 year Sharpe Ratios
sharpe_ratios_combined = sharpe_ratios_5y.merge(sharpe_ratios_10y, on='Ticker')
sharpe_ratios_combined = sharpe_ratios_combined.merge(sharpe_ratios_15y, on='Ticker')

# Rename the Sharpe Ratio columns to specify the timeframes
sharpe_ratios_combined.rename(columns={
    'Sharpe Ratio': 'Sharpe Ratio (5 years)',
    'Sharpe Ratio_x': 'Sharpe Ratio (10 years)',
    'Sharpe Ratio_y': 'Sharpe Ratio (15 years)'
}, inplace=True)

sharpe_ratios_combined.set_index("Ticker", inplace = True)

sharpe_ratios_combined

,Sharpe Ratio (5 years),Sharpe Ratio (10 years),Sharpe Ratio (15 years)
Ticker,,,
V,46.903904,51.466142,43.647353
JNJ,25.256420,35.474155,85.049696
HSY,20.011142,39.191539,37.633397
S&P 500,18.262450,31.076942,83.952682
CROX,9.717318,11.988586,17.349017
JWN,9.501606,14.234929,15.235803
AMD,6.953113,8.541401,9.103983


In [15]:
sharpe_ratios_combined.hvplot(
    xlabel="Ticker", 
    ylabel="Sharpe Ratios",
    title='Sharpe Ratios (5, 10 and 15 years)',
    width = 1800
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

# Volume 

In [16]:
# Calculate the total overall volume for each ticker
ticker_total_volume = us_stock_prices_df.groupby('Ticker')['Volume'].sum()

# Sort tickers based on total volume in descending order
sorted_tickers = ticker_total_volume.sort_values(ascending=False)

# Get the top N tickers with the most volume
top_n_tickers = 10  # Change this value to the desired number of top tickers
top_tickers = sorted_tickers.head(top_n_tickers)
top_tickers.columns = ['Ticker', 'Volume']

# Print the list of top tickers with the most volume
print("Top", top_n_tickers, "Tickers with the Most Volume:")
print(top_tickers)


Top 10 Tickers with the Most Volume:
Ticker
TSLA    3.210157e+11
AMD     1.953171e+11
V       5.910629e+10
JNJ     5.448871e+10
SQ      2.113475e+10
JWN     1.911197e+10
PINS    1.397512e+10
PTON    1.258225e+10
CROX    9.783507e+09
HSY     7.081999e+09
Name: Volume, dtype: float64


In [17]:
top_tickers.hvplot(
    xlabel="Ticker", 
    ylabel='Volume',
    title='Top 10 Tickers with the most Volume',
    width = 1000
).opts(yformatter='%.0f')

:Curve   [Ticker]   (Volume)

In [18]:
#### Find the total volume for 5, 10 and 15 year time frame

#  Create a function to calculate total volume for the top N tickers in a given time frame
def calculate_top_tickers_total_volume(data_df, time_frame, top_n_tickers=10):
    # Filter data for the given time frame
    data_df_time_frame = data_df.groupby('Ticker').tail(time_frame)
    
    # Calculate the total volume for each ticker in the time frame
    ticker_total_volume = data_df_time_frame.groupby('Ticker')['Volume'].sum()
    
    # Sort tickers based on total volume in descending order
    sorted_tickers = ticker_total_volume.sort_values(ascending=False)
    
    # Get the top N tickers with the most volume
    top_tickers = sorted_tickers.head(top_n_tickers)
    
    return top_tickers

# Calculate top N tickers' total volume for the specified time frames
top_n_tickers = 10  # Change this value to the desired number of top tickers
time_frames = [1, 5, 10, 15]  # List of time frames (in years)

# Create an empty list to store the DataFrames for each ticker
v_results_dfs = []

# List of unique tickers
tickers = combined_df['Ticker'].unique()

# Loop through each unique ticker
for ticker in tickers:
    ticker_data = combined_df[combined_df['Ticker'] == ticker].copy()
    
    # Calculate total volume for each time frame
    volume_by_time_frame = []
    for time_frame in time_frames:
        top_tickers = calculate_top_tickers_total_volume(ticker_data, time_frame, top_n_tickers)
        total_volume = top_tickers.sum()
        volume_by_time_frame.append(total_volume)
    
    # Create a dictionary to store the results for this ticker
    ticker_results = {'Ticker': ticker}
    for i, years in enumerate(time_frames):
        ticker_results[f'{years}-Year Volume'] = volume_by_time_frame[i]
    
    # Convert the results for this ticker to a DataFrame and append it to the list
    v_results_dfs.append(pd.DataFrame([ticker_results]))

# Concatenate the list of DataFrames into a single DataFrame
v_results_df = pd.concat(v_results_dfs, ignore_index=True)

# Print the DataFrame with results
print(v_results_df)

     Ticker  1-Year Volume  5-Year Volume  10-Year Volume  15-Year Volume
0      PTON   6.689800e+06   4.118870e+07    1.694141e+08    2.106857e+08
1   S&P 500   3.865960e+09   1.825606e+10    3.535253e+10    5.626857e+10
2      TSLA   1.294696e+08   6.266398e+08    1.172379e+09    1.765296e+09
3       HSY   1.087200e+06   6.128600e+06    1.180450e+07    1.825890e+07
4      PINS   1.104040e+07   4.047480e+07    7.368040e+07    1.316741e+08
5      CROX   1.079200e+06   4.821400e+06    1.185790e+07    2.248400e+07
6       AMD   6.604220e+07   2.854677e+08    6.268181e+08    9.032298e+08
7         V   4.459600e+06   2.265290e+07    4.330300e+07    6.238770e+07
8        SQ   5.987000e+06   3.326000e+07    7.699840e+07    1.242544e+08
9       JNJ   9.019800e+06   5.511000e+07    3.383554e+08    7.849871e+08
10      JWN   5.640200e+06   2.639850e+07    7.931100e+07    1.015015e+08


In [19]:
v_results_df.set_index("Ticker", inplace = True)
v_results_df.hvplot(
    xlabel="Ticker", 
    ylabel="Volume",
    title='Volume (5, 10 and 15 years)',
    width = 2500
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

# Annualized and cumulative Returns with S&P 500 as comparison 

## Annualized returns over 5, 10 and 15 years

In [20]:
# Define the time frames in trading days (1, 5, 10, and 15 years)
time_frames = [252, 5 * 252, 10 * 252, 15 * 252]

# Create an empty list to store the DataFrames for each ticker
a_results_dfs = []

# List of unique tickers
tickers = combined_df['Ticker'].unique()

# Loop through each unique ticker
for ticker in tickers:
    ticker_data = combined_df[combined_df['Ticker'] == ticker].copy()
    
    # Calculate annualized returns for each time frame
    returns_by_time_frame = []
    for time_frame in time_frames:
        data_time_frame = ticker_data.tail(time_frame)
        daily_returns = data_time_frame['Close'].pct_change()
        annualized_return = (1 + daily_returns.mean()) ** 252 - 1
        returns_by_time_frame.append(annualized_return)
    
    # Create a dictionary to store the results for this ticker
    ticker_results = {'Ticker': ticker}
    for i, years in enumerate(time_frames):
        column_name = f'{years}-Year Annualized Return'
        ticker_results[column_name] = returns_by_time_frame[i]
    
    # Convert the results for this ticker to a DataFrame and append it to the list
    a_results_dfs.append(pd.DataFrame([ticker_results]))

# Concatenate the list of DataFrames into a single DataFrame
a_results_df = pd.concat(a_results_dfs, ignore_index=True)

# Rename the columns for easier reading
a_results_df.columns = ['Ticker', '1-Year', '5-Year', '10-Year', '15-Year']

# Print the DataFrame with results
print(a_results_df)

     Ticker    1-Year    5-Year   10-Year   15-Year
0      PTON  0.030938  1.458144  1.458144  1.458144
1   S&P 500  0.195820  0.106480  0.114448  0.112498
2      TSLA  0.141223  1.069723  0.599941  0.597144
3       HSY -0.020644  0.217258  0.136754  0.178439
4      PINS  0.476941  0.273947  0.273947  0.273947
5      CROX  0.645649  0.662218  0.409681  0.532682
6       AMD  0.587157  0.579193  0.676779  0.447839
7         V  0.277811  0.163547  0.234475  0.245437
8        SQ  0.066417  0.139205  0.339884  0.339884
9       JNJ  0.030429  0.086857  0.111045  0.105245
10      JWN  0.058718 -0.031643  0.041759  0.119142


In [21]:
a_results_df.set_index("Ticker", inplace = True)
a_results_df.hvplot(
    xlabel="Ticker", 
    ylabel="Annualized Returns",
    title='Annualized Returns (1, 5, 10 and 15 years)',
    width = 2500
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

## Cumulative Returns over 1, 5, 10 and 15 years

In [22]:
# Define the time frames in trading days (1, 5, 10, and 15 years)
time_frames = [252, 5 * 252, 10 * 252, 15 * 252]

# Create an empty list to store the DataFrames for each ticker
c_results_dfs = []

# List of unique tickers
tickers = combined_df['Ticker'].unique()

# Loop through each unique ticker
for ticker in tickers:
    ticker_data = combined_df[combined_df['Ticker'] == ticker].copy()
    
    # Calculate cumulative returns for each time frame
    returns_by_time_frame = []
    for time_frame in time_frames:
        data_time_frame = ticker_data.tail(time_frame)
        cumulative_return = (1 + data_time_frame['Close'].pct_change()).prod() - 1
        returns_by_time_frame.append(cumulative_return)
    
    # Create a dictionary to store the results for this ticker
    ticker_results = {'Ticker': ticker}
    for i, years in enumerate(time_frames):
        column_name = f'{years}-Year Cumulative Return'
        ticker_results[column_name] = returns_by_time_frame[i]
    
    # Convert the results for this ticker to a DataFrame and append it to the list
    c_results_dfs.append(pd.DataFrame([ticker_results]))

# Concatenate the list of DataFrames into a single DataFrame
c_results_df = pd.concat(c_results_dfs, ignore_index=True)

# Rename the columns for easier reading
c_results_df.columns = ['Ticker', '1-Year', '5-Year', '10-Year', '15-Year']

# Print the DataFrame with results
print(c_results_df)



     Ticker    1-Year     5-Year    10-Year    15-Year
0      PTON -0.301907   0.411135   0.411135   0.411135
1   S&P 500  0.177884   0.471534   1.524357   2.615953
2      TSLA -0.050775  11.753928  21.765382  -0.023230
3       HSY -0.035803   1.336837   1.877684   7.274317
4      PINS  0.304033   0.065137   0.065137   0.065137
5      CROX  0.385775   3.762343   6.309807  22.205190
6       AMD  0.380608   3.401271  32.877676  16.724800
7         V  0.250579   0.727595   5.033053  13.430947
8        SQ -0.114428  -0.337319   0.234812   0.234812
9       JNJ  0.015735   0.363630   1.438714   2.501328
10      JWN -0.096436  -0.722517  -0.618233  -0.215165


In [23]:
c_results_df.set_index("Ticker", inplace = True)
c_results_df.hvplot(
    xlabel="Ticker", 
    ylabel="Cumulative Returns",
    title='Cumulative Returns (1, 5, 10 and 15 years)',
    width = 2500
).opts(yformatter='%.0f')

:NdOverlay   [Variable]
   :Curve   [Ticker]   (value)

In [24]:
#Calculate the average sma and prepare for comparison
meansummary=sma_summary_df[sma_summary_df['Ticker'] != 'S&P 500'].dropna()
summarySPmean= sma_summary_df[sma_summary_df['Ticker'] == 'S&P 500'].dropna()
summarySPmean1= summarySPmean['SMA_1260'].mean()
summarySPmean2= summarySPmean['SMA_2520'].mean()
summarySPmean3= summarySPmean['SMA_3780'].mean()
meansummary1=meansummary['SMA_1260'].mean()
meansummary2=meansummary['SMA_2520'].mean()
meansummary3=meansummary['SMA_3780'].mean()

#Create a new sma dataframe
smadata= {
    'SMA_1260': [meansummary1, summarySPmean1],
    'SMA_2520': [meansummary2, summarySPmean2],
    'SMA_3780': [meansummary3, summarySPmean3]
}

SMA_finale= pd.DataFrame(smadata, index=['Market', 'S&P 500'])

In [25]:
#Calculate the average sharperatio and prepare for comparison
meansharpe=sharpe_ratios_combined[sharpe_ratios_combined.index != 'S&P 500'].dropna()
SharpeSPmean= sharpe_ratios_combined[sharpe_ratios_combined.index == 'S&P 500'].dropna()
SharpeSPmean1= SharpeSPmean['Sharpe Ratio (5 years)'].mean()
SharpeSPmean2= SharpeSPmean['Sharpe Ratio (10 years)'].mean()
SharpeSPmean3= SharpeSPmean['Sharpe Ratio (15 years)'].mean()
meansharpe1=meansharpe['Sharpe Ratio (5 years)'].mean()
meansharpe2=meansharpe['Sharpe Ratio (10 years)'].mean()
meansharpe3=meansharpe['Sharpe Ratio (15 years)'].mean()

#Create a new sma dataframe
sharpedata= {
    'Sharpe Ratio (5 years)': [meansharpe1, SharpeSPmean1],
    'Sharpe Ratio (10 years)': [meansharpe2, SharpeSPmean2],
    'Sharpe Ratio (15 years)': [meansharpe3, SharpeSPmean3]
}

Sharpe_finale= pd.DataFrame(sharpedata, index=['Market', 'S&P 500'])

In [26]:
#Calculate the average volume and prepare for comparison
meanvol=v_results_df[v_results_df.index != 'S&P 500'].dropna()
SPvolmean= v_results_df[v_results_df.index == 'S&P 500'].dropna()
SPvolmean1= SPvolmean['1-Year Volume'].mean()
SPvolmean2= SPvolmean['5-Year Volume'].mean()
SPvolmean3= SPvolmean['10-Year Volume'].mean()
SPvolmean4= SPvolmean['15-Year Volume'].mean()
meanvol1= meanvol['1-Year Volume'].mean()
meanvol2= meanvol['5-Year Volume'].mean()
meanvol3= meanvol['10-Year Volume'].mean()
meanvol4= meanvol['15-Year Volume'].mean()


#Create a new sma dataframe
voldata= {
    '1-Year Volume': [meanvol1, SPvolmean1],
    '5-Year Volume': [meanvol2, SPvolmean2],
    '10-Year Volume': [meanvol3, SPvolmean3],
    '15-Year Volume': [meanvol4, SPvolmean4]
}

Vol_finale= pd.DataFrame(voldata, index=['Market', 'S&P 500'])

In [27]:
#Calculate the average volume and prepare for comparison
meanaccum=a_results_df[a_results_df.index != 'S&P 500'].dropna()
SPaccum= a_results_df[a_results_df.index == 'S&P 500'].dropna()
SPaccum1= SPaccum['1-Year'].mean()
SPaccum2= SPaccum['5-Year'].mean()
SPaccum3= SPaccum['10-Year'].mean()
SPaccum4= SPaccum['15-Year'].mean()
meanaccum1= meanaccum['1-Year'].mean()
meanaccum2= meanaccum['5-Year'].mean()
meanaccum3= meanaccum['10-Year'].mean()
meanaccum4= meanaccum['15-Year'].mean()


#Create a new sma dataframe
accdata= {
    '1-Year': [meanaccum1, SPaccum1],
    '5-Year': [meanaccum2, SPaccum2],
    '10-Year': [meanaccum3, SPaccum3],
    '15-Year': [meanaccum4, SPaccum4]
}

Annualized_finale= pd.DataFrame(accdata, index=['Market', 'S&P 500'])

In [28]:
#Calculate the average volume and prepare for comparison
meanret=c_results_df[c_results_df.index != 'S&P 500'].dropna()
SPretmean= c_results_df[c_results_df.index == 'S&P 500'].dropna()
SPretmean1= SPretmean['1-Year'].mean()
SPretmean2= SPretmean['5-Year'].mean()
SPretmean3= SPretmean['10-Year'].mean()
SPretmean4= SPretmean['15-Year'].mean()
meanret1= meanret['1-Year'].mean()
meanret2= meanret['5-Year'].mean()
meanret3= meanret['10-Year'].mean()
meanret4= meanret['15-Year'].mean()


#Create a new sma dataframe
retdata= {
    '1-Year': [meanaccum1, SPaccum1],
    '5-Year': [meanaccum2, SPaccum2],
    '10-Year': [meanaccum3, SPaccum3],
    '15-Year': [meanaccum4, SPaccum4]
}

Cum_Ret_finale= pd.DataFrame(retdata, index=['Market', 'S&P 500'])

In [29]:
#Store Long-Term Variables
#Cumulative Returns Dataframe
%store Cum_Ret_finale

#Annualized Returns Dataframe
%store Annualized_finale

#Total Volume comparison Dataframe
%store Vol_finale

#Combined Sharpe Ratios
%store Sharpe_finale

#Best SMA Results
%store SMA_finale

Stored 'Cum_Ret_finale' (DataFrame)
Stored 'Annualized_finale' (DataFrame)
Stored 'Vol_finale' (DataFrame)
Stored 'Sharpe_finale' (DataFrame)
Stored 'SMA_finale' (DataFrame)
